# Hardware
### DGX Station
V100 SMX2 16GB

Intel Xeon E5-2968 v4 @ 2.20GHz

# Software
CUDA 11.0

cuSignal 0.19

SciPy 1.6.3

Numba 0.53.1

## CPU Baseline
Baseline Scipy's Signal Lombscargle function

In [ ]:
!nsys profile -s none -o scipy_v1 -f true --stats=true python3 icassp_scipy_v1.py float32 5

In [ ]:
!nsys profile -s none -o scipy_v1 -f true --stats=true python3 icassp_scipy_v1.py float64 5

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

In [ ]:
!nsys profile -s none -o cupy_v1 -f true --stats=true python3 icassp_cupy_v1.py float32 2000 0

In [ ]:
#!nsys stats --report gpukernsum --report nvtxppsum --report gputrace cupy_v1.qdrep # Use to get register usage
!nsys stats --report gpukernsum --report nvtxppsum cupy_v1.qdrep

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum --report gputrace cupy_v1.qdrep # Use to get register usage

In [ ]:
!nsys profile -s none -o cupy_v1 -f true python3 icassp_cupy_v1.py float64 2000 0

In [ ]:
#!nsys stats --report gpukernsum --report nvtxppsum --report gputrace cupy_v1.qdrep # Use to get register usage
!nsys stats --report gpukernsum --report nvtxppsum cupy_v1.qdrep

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum --report gputrace cupy_v1.qdrep # Use to get register usage

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

In [ ]:
!nsys profile -s none -o cupy_v2 -f true python3 icassp_cupy_v2.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v2.qdrep

In [ ]:
!nsys profile -s none -o cupy_v2 -f true python3 icassp_cupy_v2.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v2.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

In [ ]:
!nsys profile -s none -o cupy_v3 -f true python3 icassp_cupy_v3.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v3.qdrep

In [ ]:
!nsys profile -s none -o cupy_v3 -f true python3 icassp_cupy_v3.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v3.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

In [ ]:
!nsys profile -s none -o cupy_v4 -f true python3 icassp_cupy_v4.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v4.qdrep

In [ ]:
!nsys profile -s none -o cupy_v4 -f true python3 icassp_cupy_v4.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v4.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

In [ ]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle.cu -odir .

In [ ]:
!nsys profile -s none -o cupy_v5 -f true python3 icassp_cupy_v5.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v5.qdrep

In [ ]:
!nsys profile -s none -o cupy_v5 -f true python3 icassp_cupy_v5.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v5.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

In [ ]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle_lb.cu -odir .

In [ ]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

In [ ]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

In [ ]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle_lb_trig.cu -odir .

In [ ]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

In [ ]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |